In [ ]:
import json
import warnings
from datasets import load_dataset

warnings.filterwarnings('ignore')

/users/k24053411/.conda/envs/fever/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def copy_leaving_out(dictionary, key):
    copy = dictionary.copy()
    del copy[key]
    return copy

def process_data(dataset): 
    
    processed_data = {}
    
    for data in dataset:
        task = data["task"]

        data_filtered = copy_leaving_out(data, "task")

        if task in processed_data.keys():
            processed_data[task].append(data_filtered)
        else:
            processed_data[task] = [data_filtered]
            
    for task in processed_data.keys():

        source_dict = {}
        dataset = processed_data[task]
        
        for data in dataset:
            source = data["source"]

            data_filtered = copy_leaving_out(data, "source")

            if source in source_dict.keys():
                source_dict[source].append(data_filtered)
            else:
                source_dict[source] = [data_filtered]
        
        processed_data[task] = source_dict
        
    for task in processed_data.keys():

        source_dict = processed_data[task]

        for source in source_dict.keys():
            
            variety_dict = {}
            dataset = source_dict[source]
            
            for data in dataset:
                variety = data["variety"]

                data_filtered = copy_leaving_out(data, "variety")

                if variety in variety_dict.keys():
                    variety_dict[variety].append(data_filtered)
                else:
                    variety_dict[variety] = [data_filtered]
            
            source_dict[source] = variety_dict
        
        processed_data[task] = source_dict
        
    print('Sanity Check : ', len(processed_data['Sentiment']['Google']["en-UK"]))
    
    return processed_data

In [ ]:
print("Loading Besstie dataset from Hugging Face...")
dataset_train = load_dataset("unswnlporg/BESSTIE", split =  "train")
dataset_test = load_dataset("unswnlporg/BESSTIE", split =  "validation")

print(f"Loaded BESSTIE dataset with {len(dataset_train)} (train) and {len(dataset_test)} (test) total samples")
print(f"Dataset columns: {list(dataset_train.features.keys())}")

Loading Besstie dataset from Hugging Face...
Loaded Dolly dataset with 2428 total samples
Dataset columns: ['text', 'label', 'variety', 'source', 'task']


In [ ]:
dataset_train[0:5]

{'text': ["Ordered red hot fried and hot honey fried. The taste is good for a fried chicken. The sauce is not coated evenly., but that's okay I guess. The size is big, that's good.",
  'I wish we had found this place for lunch, but instead, we tried one of the other places but stopped here for Bingsu. From what we did see, the food looks like a winner here, so I will be back to try on the next trip. Serving was decent and a great concept for a restaurant with various Asian Cuisine options.',
  "If you're in the mood for a culinary adventure, look no further than Starfish. Nestled alongside a picturesque riverside patio, this gem of a bistro offers an exquisite selection of seafood and wood-fired pizza that will leave your taste buds dancing with delight. The standout for me was the delectable Thai Chicken Pizza. The harmonious blend of zesty Thai flavors with succulent chicken and perfectly baked crust was nothing short of a taste revelation. It's a bold fusion that pays off in spades-

In [ ]:
processed_data = process_data(dataset_train)

with open('../data/instruction/besstie/train.json', 'w') as f:
    json.dump(processed_data, f)

In [ ]:
processed_data = process_data(dataset_test)

with open('../data/instruction/besstie/test.json', 'w') as f:
    json.dump(processed_data, f)

248